# 데일리차트 접속 후 인기 포메이션, 인기 팀 컬러 조사하기

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import chromedriver_autoinstaller
import sys
import os
import re
import requests
import pandas as pd
import numpy as np
import time

chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)
url = 'https://fifaonline4.nexon.com/datacenter/dailysquad'
driver.get(url)
response = requests.get(url)
html = response.text

C:\Users\user\AppData\Local\Temp/ipykernel_16460/2310303632.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)


In [2]:
contents = driver.find_elements(By.CSS_SELECTOR,'#formationSwiper > div > div > div > div')
len(contents)

10

In [3]:
contents[0].find_element(By.CSS_SELECTOR,'div.txt').text # 잘 나오는 것 확인

'4-2-3-1'

In [4]:
formation = []
for content in contents:
    formae = content.find_element(By.CSS_SELECTOR,'div.txt').text
    formation.append(formae)
    
formation_top6 = formation[:6] # 상위 6개 포메이션 선정
print(formation_top6)

['4-2-3-1', '4-2-2-1-1', '5-2-3', '4-2-2-2', '4-2-1-3', '5-2-1-2']


In [5]:
formation_top4 = ['4-2-3-1','5-2-3','4-1-2-3','5-2-1-2'] # 시스템 구현 시엔 이 리스트를 사용할 예정

In [6]:
teams = driver.find_elements(By.CSS_SELECTOR,'#divTeamColorInfo > div')
print(len(teams))

10


In [7]:
print(teams[0].find_element(By.CSS_SELECTOR,'div.txt').text)

첼시


In [8]:
team_color=[]

for team in teams:
    tc = team.find_element(By.CSS_SELECTOR,'div.txt').text
    team_color.append(tc)
    
team_color_top4 = team_color[:4] # 상위 4개 팀컬러 선정
print(team_color_top4)

['첼시', '대한민국', '레알 마드리드', '바이에른 뮌헨']


In [9]:
team_color_top2 = ['대한민국','바이에른 뮌헨'] # 실제 시스템 구현 시 이 리스트 사용 예정, 시간 단축 목적

In [10]:
driver.quit()

In [11]:
PlayersData = pd.read_csv("./players_data/players_data.csv",encoding='cp949') # 추출한 선수 명단 데이터를 불러온다.

In [12]:
PlayersData

,포지션,이름,클래스
0,ST,박주영,2012KH
1,ST,R. 레반도프스키,22TOTY
2,ST,크리스티아누 호날두,22TOTS
3,ST,호나우두,MC
4,ST,크리스티아누 호날두,21TOTY
...,...,...,...
235,GK,에우통 레이치,21UCL
236,GK,닉 포프,LIVE
237,GK,조 하트,TC
238,GK,보이치에흐 슈체스니,EBS


In [13]:
PlayersData_list = PlayersData.values.tolist()
print(type(PlayersData_list))
print(PlayersData_list[20:50])

<class 'list'>
[['CF', '차범근', 'LH'], ['CF', '세르주 그나브리', '21UCL'], ['CF', '카이 하베르츠', '21UCL'], ['CF', '손흥민', '22TOTY'], ['CF', '고정운', 'TKL'], ['CF', '이반 페리시치', '21UCL'], ['CF', '카림 벤제마', 'BOE21'], ['CF', '메이슨 마운트', '22TOTN'], ['CF', '호나우두', 'MC'], ['CF', '손흥민', '21TOTY'], ['CF', '카림 벤제마', '21UCL'], ['CF', '모하메드 살라', '22TOTS'], ['LW', '사디오 마네', '21UCL'], ['LW', '에덴 아자르', 'EBS'], ['LW', '고정운', 'TKL'], ['LW', '필 포든', '22TOTS'], ['LW', '차범근', 'LH'], ['LW', '비니시우스 주니어', '21UCL'], ['LW', '페드로', 'UP'], ['LW', '세르주 그나브리', '21UCL'], ['LW', '비니시우스 주니어', '21NG'], ['LW', '프랑크 리베리', 'VTR'], ['LW', '비니시우스 주니어', '22TOTS'], ['LW', '네이마르 Jr.', '22TOTN'], ['LW', '사디오 마네', 'MC'], ['LW', '필 포든', '22TOTN'], ['LW', '이반 페리시치', '21UCL'], ['LW', '나니', 'VTR'], ['RW', '차범근', 'LH'], ['RW', '세르주 그나브리', '21UCL']]


# 스쿼드메이커를 이용한 자동 스쿼드 제작 시스템

In [14]:
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path) 
url = 'https://fifaonline4.nexon.com/datacenter/squadmaker'
driver.get(url)

C:\Users\user\AppData\Local\Temp/ipykernel_16460/304761245.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)


## FO4 모든 클럽 팀컬러 리스트로 수집

In [15]:
squad = driver.find_element(By.CSS_SELECTOR,'#formationPlayer11')
squad.click()
time.sleep(2)
wrap_teamcolor = '#squadSearchform1 > div > div.search_form > div > div.search_input_detail > div.search_input_detail_teamcolor'
search_teamcolor = driver.find_element(By.CSS_SELECTOR, wrap_teamcolor+'>div.wrap_teamcolor')

club_teamcolor = ' div > ul > li:nth-child(2)'
search_teamcolor.click()
time.sleep(2)
club_searching = search_teamcolor.find_element(By.CSS_SELECTOR,club_teamcolor) # 클럽 선택
club_searching.click()
time.sleep(2)
club_colorlists = driver.find_elements(By.CSS_SELECTOR, wrap_teamcolor+'> div.wrap_teamcolorlist > div > ul > li')


total_ClubColorList = [] # 피온4에서 제공하는 모든 클럽 팀컬러, 찾고자 하는 팀컬러가 여기에 없으면 국가 팀컬러라고 전제한다.
for c in range(len(club_colorlists)):
    club = club_colorlists[c].text
    total_ClubColorList.append(club)
while '' in total_ClubColorList:
    total_ClubColorList.remove('')
total_ClubColorList = total_ClubColorList[1:]    

total_ClubColorList[:20]

['맨체스터 시티',
 '리버풀',
 '토트넘 홋스퍼',
 '첼시',
 '아스널',
 '맨체스터 유나이티드',
 '울버햄프턴 원더러스',
 '레스터 시티',
 '에버턴',
 '왓퍼드',
 '바이에른 뮌헨',
 '보루시아 도르트문트',
 'RB 라이프치히',
 '아인트라흐트 프랑크푸르트',
 '바이엘 04 레버쿠젠',
 'VfL 볼프스부르크',
 'FC 바르셀로나',
 '아틀레티코 마드리드',
 '레알 마드리드',
 '발렌시아 CF']

In [16]:
driver.quit()

## 로그인 진행(스쿼드메이커 저장 작업 시 로그인이 되어있어야 함)

In [17]:
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path) 
url = 'https://fifaonline4.nexon.com/datacenter/squadmaker'
driver.get(url)
time.sleep(2)
driver.maximize_window()

C:\Users\user\AppData\Local\Temp/ipykernel_16460/2129433919.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)


In [18]:
from getpass import getpass

nexon_ID = input('아이디를 입력하세요: ')
nexon_PW = getpass('비밀번호를 입력하세요: ')
time.sleep(2)

login_btn = driver.find_element(By.CSS_SELECTOR,'body > div.gnbWrapper.gnbSizeL > div.gnbBar > div.gnbBarRight > div.gnbLogin').click()
time.sleep(2)
alert = driver.switch_to.alert
#time.sleep(2)
alert.accept() # 경고창 뜨는 거 방지

ID_input = driver.find_element(By.CSS_SELECTOR,'#txtNexonID').send_keys(nexon_ID) # 넥슨 개인 ID
pwd_input = driver.find_element(By.CSS_SELECTOR,'#txtPWD').send_keys(nexon_PW) # 넥슨 개인 PW
time.sleep(1)
click_loginbtn = driver.find_element(By.CSS_SELECTOR,'#nexonLogin > fieldset > div.btLogin > button').click()

아이디를 입력하세요: khpoptop@gmail.com
비밀번호를 입력하세요: ········


### 포지션 정하기
> PlayersData 상에 있는 포지션

1. LS,RS,ST,LF,RF,CF -> PlayersData의 ST,CF에서 검색
    - Forward
2. LW,RW,LM,RM,LAM,RAM -> PlayersData의 LW,RW,LM,RM에서 검색
    - Winger
3. CAM -> PlayersData의 CAM에서 검색
    - Attack_mid
4. LCM,RCM,CM -> PlayersData의 CM,CAM에서 검색
    - Central_mid
5. LDM,RDM,DM -> PlayersData의 CDM,CM에서 검색
    - Defend_mid
6. LB,LWB -> PlayersData의 LB,LWB에서 검색
    - Leftback
7. RB,RWB -> PlayersData의 RB,RWB에서 검색
    - Rightback
8. LCB,RCB,CB -> PlayersData의 CB에서 검색
    - Centerback
9. GK는 GK에서 검색
    - Goalkeeper

In [19]:
driver.execute_script("window.scrollTo(0, 750)") 

In [20]:
def choose_player():   
    for player in PlayersData_sorted:
        player_input = driver.find_element(By.CSS_SELECTOR,'#searchName').send_keys(player[1])
        time.sleep(1)
        player_searchbtn = driver.find_element(By.CSS_SELECTOR,'#squadSearchPlayerBtn').click()
        time.sleep(2)
        search_result = driver.find_elements(By.CSS_SELECTOR,'#ulPlayerList > li > div > div.info_top > div.name') # 검색 결과
        #checked = driver.find_element(By.XPATH,'//*[@id="checkbox1"]').get_attribute('checked') # 스쿼드에 미리 포함되어있는지 확인
        if len(search_result)==0 or driver.find_element(By.XPATH,'//*[@id="checkbox1"]').get_attribute('checked')=='true':
            continue
        else:
            player_name = driver.find_elements(By.CSS_SELECTOR,'#ulPlayerList > li > div > div.info_top > div.name')[0].text
            player_searched = driver.find_elements(By.CSS_SELECTOR,'#ulPlayerList > li') # 검색한 선수 목록
            for k in range(len(player_searched)):
                divTag = driver.find_element(By.CSS_SELECTOR,'#ulPlayerList > li.player.player'+str(k)+' > div > div.info_top > div.season > img')
                src = divTag.get_attribute('src')
                regex = re.compile('{}(.*){}'.format(re.escape('season/'), re.escape('.png')))
                player_season = regex.findall(str(src))[0]
                if player_season!=player[2]:
                    continue
                else:
                    if player[1]==player_name: # 시즌, 이름 모두 같으면 선수를 스쿼드에 포함
                        player_searched[k].click()
                        time.sleep(2)
                        break
                    else:
                        continue
                    break
        if PlayersData_sorted[-1][1]!=player_name:
            break
        break            

In [21]:
column_len = driver.find_elements(By.CSS_SELECTOR,'div > div.squadmaker-selector__list__contents > div.squadmaker-selector__list.active > div.squadmaker-selector__list__total>div')
index_len = column_len[0].find_elements(By.CSS_SELECTOR,'ul > li > a')

formation_btn = driver.find_element(By.CSS_SELECTOR,'#squadMaker > div > div.squadmaker__utils > div.squadmaker__utils__aside__selector')
formation_total = [] # 셀레니움 형태로 저장, 내용 볼겨면 .text 붙혀서 확인
formation_btn.click()
for i in range(1,len(column_len)+1):
    index_len = column_len[i-1].find_elements(By.CSS_SELECTOR,'ul > li > a')
    for j in range(1,len(index_len)+1):
        time.sleep(2)     
        formae = driver.find_element(By.CSS_SELECTOR,'#squadMaker > div > div.squadmaker__utils > div.squadmaker__utils__aside__selector.squadmaker-selector.active > div > div.squadmaker-selector__list__contents > div.squadmaker-selector__list.active > div.squadmaker-selector__list__total > div:nth-child('+str(i)+') > ul > li:nth-child('+str(j)+')')
        formation_total.append(formae)
    print('column {0} complete'.format(i))
    
print(formation_total[2].text)
print(formation_total[11].text)
    
formation_btn.click()

column 1 complete
column 2 complete
column 3 complete
column 4 complete
column 5 complete
column 6 complete
column 7 complete
3-4-1-2
4-3-3


In [22]:
# 실행부분

wrap_teamcolor = '#squadSearchform1 > div > div.search_form > div > div.search_input_detail > div.search_input_detail_teamcolor'
club_teamcolor = ' div > ul > li:nth-child(2)'
nation_teamcolor = 'div > ul > li:nth-child(3)'

formation_btn.click()
for formae in formation_total:
    if formae.text in formation_top4:
        #print('지정된 포메이션: ',formae.text)
        time.sleep(2)
        formae.click() # 화면 임의로 축소하면 버그 발생하므로 건드리지 말 것!
        time.sleep(2)
        num = 1
        for team_color in team_color_top2: # 지정된 2개의 팀컬러 중에서 순차적으로 선택
            driver.execute_script("window.scrollTo(0, 930)")
            time.sleep(2)
            for j in range(11,0,-1):
                position_name = driver.find_element(By.CSS_SELECTOR,'#formationPlayer'+str(j)+' > span > span.info > span.position').text
                PlayersData_sorted=[]
                for playersdata in PlayersData_list: # 선수 데이터 리스트
                    if position_name=='LS' or position_name=='RS' or position_name=='ST' or position_name=='LF' or position_name=='RF' or position_name=='CF':
                        if playersdata[0]=='ST' or playersdata[0]=='CF':
                            PlayersData_sorted.append(playersdata)
                    elif position_name=='LW' or position_name=='RW' or position_name=='LM' or position_name=='RM' or position_name=='LAM' or position_name=='RAM':
                        if playersdata[0]=='LW' or playersdata[0]=='RW' or playersdata[0]=='LM' or playersdata[0]=='RM':
                            PlayersData_sorted.append(playersdata)
                    elif position_name=='CAM':
                        if playersdata[0]=='CAM':
                            PlayersData_sorted.append(playersdata)
                    elif position_name == 'LCM' or position_name=='RCM' or position_name=='CM':
                        if playersdata[0]=='CM' or playersdata[0]=='CAM':
                            PlayersData_sorted.append(playersdata)
                    elif position_name=='LDM' or position_name=='RDM' or position_name=='CDM':
                        if playersdata[0]=='CDM' or playersdata[0]=='CM':
                            PlayersData_sorted.append(playersdata)
                    elif position_name=='LB' or position_name=='LWB':
                        if playersdata[0]=='LB' or playersdata[0]=='LWB':
                            PlayersData_sorted.append(playersdata)
                    elif position_name=='RB' or position_name=='RWB':
                        if playersdata[0]=='RB' or playersdata[0]=='RWB':
                            PlayersData_sorted.append(playersdata)
                    elif position_name=='LCB' or position_name=='RCB' or position_name=='CB':
                        if playersdata[0]=='CB':
                            PlayersData_sorted.append(playersdata)
                    elif position_name=='GK':
                        if playersdata[0]=='GK':
                            PlayersData_sorted.append(playersdata)
                    else:
                        continue
                squad = driver.find_element(By.CSS_SELECTOR,'#formationPlayer'+str(j)+'> span > span.animate') 
                squad.click() # 스쿼드 십자 모양 클릭
                time.sleep(2)
                search_teamcolor = driver.find_element(By.CSS_SELECTOR, wrap_teamcolor+'>div.wrap_teamcolor')
                if team_color not in total_ClubColorList: # 클럽 팀 컬러에 없다면 국가 팀 컬러이다
                    search_teamcolor.click()
                    time.sleep(2)
                    nation_searching = search_teamcolor.find_element(By.CSS_SELECTOR,nation_teamcolor)
                    nation_searching.click()
                    time.sleep(2)
                    nation_colorlists = driver.find_elements(By.CSS_SELECTOR, wrap_teamcolor+'> div.wrap_teamcolorlist > div > ul > li')
                    for nation in nation_colorlists:
                        if nation.text==team_color:
                            nation.click()
                            time.sleep(2)
                            choose_player() # 함수 실행

                        else:
                            continue
                        break
                else:
                    search_teamcolor.click()
                    time.sleep(2)
                    club_searching = search_teamcolor.find_element(By.CSS_SELECTOR,club_teamcolor)
                    club_searching.click()
                    time.sleep(2)
                    club_colorlists = driver.find_elements(By.CSS_SELECTOR, wrap_teamcolor+'> div.wrap_teamcolorlist > div > ul > li')
                    for club in club_colorlists:
                        if club.text==team_color:
                            club.click()
                            time.sleep(2)
                            choose_player() # 함수 실행
                        else:
                            continue
                        break
                time.sleep(1)
            driver.execute_script("window.scrollTo(0,0)")
            time.sleep(1)
            current_formation = driver.find_element(By.CSS_SELECTOR,'#squadMaker > div > div.squadmaker__utils > div.squadmaker__utils__aside__selector.squadmaker-selector > a').text
            save_squad_naming = driver.find_element(By.CSS_SELECTOR,'#squadSaveName').send_keys(team_color+current_formation)
            save_squad_btn = driver.find_element(By.CSS_SELECTOR,'#middle > div > div > div.squad__wrap > div.squad__header-utils > div.squad-save-panel > div.squad-save-name-submit > button.squad-save-panel__btn-save').click()
            time.sleep(3)
            alert.accept()
            time.sleep(3)
            init_squad = driver.find_element(By.CSS_SELECTOR,'#middle > div > div > div.squad__wrap > div.squad__header-utils > div.squad-save-panel > div.squad-save-name-submit > button.squad-save-panel__btn-reset').click()
            time.sleep(3)
            alert.accept()
            time.sleep(3)
            driver.execute_script("window.scrollTo(0, 750)") 
            time.sleep(2)
            formation_btn.click()
            time.sleep(2)
            if len(team_color_top2)!=num:
                formae.click() # 초기화를 누르면 포메이션도 같이 초기화되므로 다시 눌러줘야 한다.
                print('계속 진행')
                num+=1
                time.sleep(1)
            else:
                break
        #formation_btn.click()
        #time.sleep(2)

계속 진행
계속 진행
계속 진행
계속 진행


In [23]:
driver.execute_script("window.scrollTo(0, 0)")
time.sleep(2)
logout_btn = driver.find_element(By.CSS_SELECTOR,'body > div.gnbWrapper.gnbSizeL > div.gnbBar > div.gnbBarRight > div.gnbLogout > a').click()
time.sleep(2)
alert = driver.switch_to.alert

alert.accept()
time.sleep(1)
driver.quit()

# 계정 로그아웃 및 driver 종료로 프로젝트 마무리